In [ ]:
%pip install langchain openai
%pip install tiktoken -q
%pip install ipywidgets
%pip install flask-cors

In [ ]:
import os
import json
import time
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate

PROXY_PORT       = 7890 
with open('config.json') as f:
    data = json.load(f)
OPENAI_API_KEY   = data['api']

In [ ]:
os.environ['http_proxy']      = f"127.0.0.1:{PROXY_PORT}"
os.environ['https_proxy']     = f"127.0.0.1:{PROXY_PORT}"
os.environ['OPENAI_API_KEY']  = OPENAI_API_KEY

template = """
    Assistant is a large language model trained by OpenAI.
    Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
    Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
    Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
 
{history}
Human: {human_input}
Assistant:
"""

prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
)
llm = ChatOpenAI(model_name="gpt-4-1106-preview", openai_api_key=OPENAI_API_KEY)
chatgpt_chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    verbose=True, 
    memory=ConversationBufferWindowMemory(k=100),
)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import json

class EditableTreeWidget(object):
    def __init__(self, prompt_json_dir=""):
        self.prompt_json_dir = prompt_json_dir
        if prompt_json_dir:
            with open(prompt_json_dir, "r") as f:
                self.prompt = json.load(f)
        else:
            self.prompt = {}
        self.initUI()

    def initUI(self):
        self.input_text = widgets.Textarea(
            value='',
            placeholder='请输入内容...',
            description='输入:',
            disabled=False,
            layout=widgets.Layout(width='80%', height='100px')
        )
        self.output_area = widgets.Output(
            layout=widgets.Layout(width='90%', min_height='100px', max_height='4000px', 
                                overflow_y='auto', border='2px solid black'),
            style={'border-radius': '5px', 'white-space': 'pre-wrap'}
        )
        
        self.input_text.observe(self.on_text_change, names='value')
        
        self.tree = self.create_tree(self.prompt)
        self.add_button = widgets.Button(description="Add Prompt")
        self.add_button.on_click(self.add_item)
        self.tree_view = widgets.VBox([self.tree, self.add_button])
        top_layout = widgets.HBox([self.tree_view, self.input_text])
        self.view = widgets.VBox([self.output_area, top_layout])
        


    def create_tree(self, prompt, level=0):
        if isinstance(prompt, dict):
            items = []
            for key, value in prompt.items():
                item_layout = self.create_tree(value, level=level+1)
                accordion = widgets.Accordion(children=[item_layout])
                accordion.set_title(0, key)
                items.append(accordion)
            return widgets.VBox(items)
        elif isinstance(prompt, list):
            items = [self.create_tree(item, level=level+1) for item in prompt]
            return widgets.VBox(items)
        else:
            text_area = widgets.Textarea(value=str(prompt), layout={'width': 'auto'})
            text_area.observe(self.on_prompt_text_change, names='value')
            return text_area

    def add_item(self, b):
        new_item = widgets.Textarea(value="New prompt", layout={'width': 'auto'})
        new_item.observe(self.on_prompt_text_change, names='value')
        self.tree.children = list(self.tree.children) + [new_item]

    def get_view(self):
        return self.view
    
    def on_text_change(self, change):
        new_text = change['new']
        if new_text.endswith('@@'):  # 假设用户按下回车键时会添加一个换行符
            current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            ans = chatgpt_chain.predict(human_input=new_text[:-2])
            ans = f"""
                <div style="background-color: #f0f0f0; border-radius: 5px; border: 0px solid black; font-family: 'Times New Roman', 宋体; font-size: 1rem; line-height: 1.12rem; padding-bottom: .5rem; padding-top: .5rem;">{current_time}</div>
                <div style="background-color: transparent; border-radius: 5px; border: 0px solid black; font-family: 'Times New Roman', 宋体; font-size: 1.2rem; line-height: 1.12rem; padding-bottom: .5rem; padding-top: .5rem;">{new_text[:-2]}</div>
                <div style="background-color: lightgreen; border-radius: 5px; border: 0px solid black; font-family: 'Times New Roman', 宋体; font-size: 1.2rem; line-height: 1.12rem; padding-bottom: .5rem; padding-top: .5rem;">{ans}</div>"""
            with self.output_area:
                display(HTML(ans))
            self.input_text.value = ""
            
    def on_prompt_text_change(self, change):
        new_text = change['new']
        if new_text.endswith('\n'):  # 假设用户按下回车键时会添加一个换行符
            self.input_text.value = new_text[:-2]
editable_tree_widget = EditableTreeWidget(prompt_json_dir="prompt.json")
editable_tree_view = editable_tree_widget.get_view()

In [ ]:
display(editable_tree_view)